# GAMALOS (GAlaxies MAking LOve Simulation)

# Theoric remarks

Report: write a short report containing:
- a short introduction to the N-body problem, the difference between
collisional and collisionless systems and the force estimate in
collisionless simulations based on tree algorithms. Short general
description of the fireworks package and its implementation.
- Description of the methods (integrators, force estimator) used.
- Description of the initial conditions and simulation setup.
- Results and discussion. They depends on the specific project you
chose, but I expect at least one plot showing the main conclusion of
your analysis.

<div class='alert alert-danger' role='alert'>
Inserisci info teoriche
</div>

# Computation

## Import the packages and the data

In [1]:
import numpy as np
import pandas as pd
from fireworks.ic import ic_two_body
import matplotlib.pyplot as plt
from fireworks.nbodylib.integrators import integrator_leapfrog_galaxy, integrator_leapfrog, integrator_leapfrog_galaxy_encounter, integrator_tsunami
from fireworks.nbodylib.dynamics import acceleration_pyfalcon
from fireworks.nbodylib.timesteps import adaptive_timestep_r
from fireworks.particles import Particles
from typing import Optional, Tuple, Callable, Union
import time

#from mpl_toolkits.mplot3d import Axes3D

%matplotlib widget

We can import the data of one galaxy at first. The units are still Nbody units.

<div class='alert alert-block alert-warning'>
occhio che ho preso 1e4 particelle!!
</div>

In [2]:
df_0 = pd.read_csv("data_cvs/Nbody_disc.csv")

# slice dataframe
num_particles = 1e4
df = df_0.iloc[:int(num_particles)]

df.head(10)

,mass,x,y,z,vx,vy,vz
0,1.000000e+00,0.000000,0.000000,0.0,0.000000,0.000000,0.0
1,1.000000e-11,-3.419583,-1.901788,0.0,0.118878,-0.213752,0.0
2,1.000000e-11,-14.046839,-2.403541,0.0,0.040952,-0.239330,0.0
3,1.000000e-11,-4.618003,6.269821,0.0,-0.222721,-0.164044,0.0
4,1.000000e-11,-6.045516,0.257974,0.0,-0.011730,-0.274897,0.0
5,1.000000e-11,-1.393104,1.578234,0.0,-0.124900,-0.110249,0.0
6,1.000000e-11,-1.150877,-1.762433,0.0,0.139480,-0.091081,0.0
7,1.000000e-11,0.593887,-1.003781,0.0,0.086283,0.051049,0.0
8,1.000000e-11,2.423492,-10.266498,0.0,0.257402,0.060762,0.0
9,1.000000e-11,-3.279328,5.118040,0.0,-0.231783,-0.148512,0.0


## Initial conditions
We generated a Particles instance for each galaxy to differentiate them. 

In [3]:
# create a class with the mass, pos, vel of each element of the Galaxy

masses = df[['mass']]
positions = df[['x', 'y', 'z']]
velocities = df[['vx', 'vy', 'vz']]

# reshape mass array as 1D array
masses = masses.values.reshape(-1)

Galaxy1 = Particles(positions.values, velocities.values, masses)

# do the same for the other galaxy
# this second galaxy is the same as the first one, but translated by a certain value

pos_translation = np.array((40., 20., 0.))
vel_translation = np.array((-0.2, 0., 0.))

# apply this translation to find the second galaxy

positions_gal_2 = positions.values + pos_translation
velocities_gal_2 = velocities.values + vel_translation
Galaxy2 = Particles(positions_gal_2, velocities_gal_2, masses)


## Rotation curve and Surface density profile
Let's plot the rotation curve and the surface density profile for one galaxy, in particular for Galaxy 1. 

The **rotation curve** of a galaxy shows the azimuthal velocity as function of the cylindrical radius . To estimate the azimuthal velocity for each particle we transform the Cartesian coordinates to cylindrical: 

$$
\begin{cases}
R_{\text{cyl}} = \sqrt(x^2+y^2) \\
\phi = \arctan(\frac{y}{x}) \\
z=z
\end{cases}
$$

We then computed the azymuthal velocity for all the particles using: 
$$
V_\phi = V_y \cdot \cos(\phi) - V_x\cdot \sin(\phi)
$$

We then plotted $V_\phi$ vs $R$ to obtain the rotation curve of the galaxy. 

The **surface density profile** $\Sigma$ shows the amount of mass in a cylindrical ring divided by the area of the ring. To estimate it we bin the particles on the cylindrical radius $R_{\text{cyl}}$, then we sum all the masses of the particles to obtain $M_{\text{tot, ring}}$ and then the area of the ring is given by $\pi (R^2_{\text{outer}}-R^2_{\text{inner}})$ where $R_{\text{inner}}$ and $R_{\text{outer}}$ are the borders of the considered bin. Finally $\Sigma_{\text{ring}}=M_{\text{tot, ring}}/A_{\text{ring}}$. 

We plotted the initial number suface density of the disc, which means that we use the number of particles instead of the total mass of the ring, as function of the cylindrical radius. 

---

## Helper functions
Let's define some helper functions: 
* `rotation_curve_rescaled`: takes as input the class and the particle we defined as the galactic center and outputs the cylindrical radius and the azimuthal velocity (rescaled to the galactic center);
* `surface_density`: computes the numerical surface density taking as input the cylindrical radius;
* `ìntegration_leapfrog`: integrates the galaxy over the time of the simulation using the integrator leapfrog, moreover computes the cyclindrical radius and the azimuthal velocity using `rotation_curve_rescaled`.

In [4]:
def rotation_curve_rescaled(galaxy: Particles, GC: float):
    
    # rescale the position and velocity of the particles
    # GC is the particle assumed as the center of the galaxy
    dx = galaxy.pos[:,0] - galaxy.pos[GC,0]
    dy = galaxy.pos[:,1] - galaxy.pos[GC,1]
    dz = galaxy.pos[:,2] - galaxy.pos[GC,2]

    dvx = galaxy.vel[:,0] - galaxy.vel[GC,0]
    dvy = galaxy.vel[:,1] - galaxy.vel[GC,1]
    dvz = galaxy.vel[:,2] - galaxy.vel[GC,2]

    # compute the cylindrical radius and azimuthal angle
    Rcyl = np.sqrt(dx**2 + dy**2)                   # cylindrical radius
    phi = np.arctan2(dy, dx)                        # azimuthal angle
    z = dz                                          # z-coordinate

    # compute azimuthal velocity    
    vel_phi = dvy * np.cos(phi) - dvx * np.sin(phi)     # azimuthal velocity

    return Rcyl, dz, vel_phi

def surface_density(Rcyl):
    H, edge = np.histogram(Rcyl, bins=20)                                          # histogram of the cylindrical radius
    area =  np.pi * (edge[1:]**2 - edge[0:-1]**2)                                  # area of every bin (every concentric ring) 
    mid = 0.5 * (edge[1:] + edge[0:-1])                                            # midpoint of every bin  

    return mid, H, area

In [5]:
def integration_leapfrog(galaxy: Particles, h: float, tsimulation: float, t: float, soft: float, GC: float):
    N = int((tsimulation - t) / h)  # number of time steps
    num_particles = len(galaxy.mass)  # number of particles in the galaxy
    path = np.empty((N, num_particles, 3))  # array to store the position of the galaxy
    velocity = np.empty((N, num_particles, 3))  # array to store the velocity of the galaxy
    time = np.empty(N)
    timestep = np.empty(N)
    
    R_cyl, _, V_phi = rotation_curve_rescaled(galaxy, GC)
    R_cyl = [R_cyl]
    V_phi = [V_phi]

    i = 0
    while t < tsimulation - h:
        updated_galaxy, _,  updated_acc, _, _ = integrator_leapfrog(particles=galaxy, tstep=h, acceleration_estimator=acceleration_pyfalcon, softening=soft)
        path[i] = updated_galaxy.pos
        velocity[i] = updated_galaxy.vel
            
        R, z, V = rotation_curve_rescaled(updated_galaxy, GC)

        timestep[i] = h
        t += h
        time[i] = t

        R_cyl.append(R)
        V_phi.append(V) 

        i += 1

    R_cyl = np.array(R_cyl)
    V_phi = np.array(V_phi)

    return path, velocity, time, R_cyl, V_phi

----

## Galaxy 1
Let's try an orbital simulation of the first galaxy to see if the integration works (in this test we used the leap-frog method).

In [6]:
# compute the orbital period with T = 2*pi*sqrt(a^3/GM)

G = 1.                                                                   # gravitational constant in n-body units
plummer_scale_lenght = 5.                                                # scale lenght of the plummer sphere (softening)

radius = np.sqrt(np.sum((Galaxy1.pos-Galaxy1.pos[0])**2., axis=1))  # radius array

central_mass = Galaxy1.mass[0]                                           # central mass of the galaxy

T = 2 * np.pi * np.sqrt(radius**3 / (G * central_mass))        # period of the galaxy using radius array  
T = np.mean(T)                                                           # mean period of the galaxy

print(f'Period of the galaxy (approximately): {T}')

# define the time of simulation and the number of steps

t0 = 0.                         # initial time
tsimulation = 1*T               # time of simulation (one period)
N = int(1e3)                    # number of steps
h = (tsimulation - t0)/N        # time steps

Galaxy1_test = Galaxy1.copy()

Galaxy1_orbit, Galaxy1_stars_vel, Galaxy1_time, Galaxy1_Rcyl, Galaxy1_Vphi = integration_leapfrog(galaxy=Galaxy1_test, h=h, 
                                                                                            tsimulation=tsimulation, t=t0, soft=plummer_scale_lenght, GC=0)

print(Galaxy1_orbit.shape)
# (iterations, particles, coordinates)

Period of the galaxy (approximately): 107.52206736218666


: 

In [ ]:
# plot of the orbit of the galaxy

plt.figure(figsize=(5,5))
plt.scatter(Galaxy1_orbit[:,1:,0], Galaxy1_orbit[:,1:,1], s=0.005, color='b', alpha=0.1)
plt.plot(Galaxy1_orbit[0,1:,0], Galaxy1_orbit[0,1:,1], 'o', markersize=0.5, c='b', label='initial position', alpha=0.5)
plt.plot(Galaxy1_orbit[-1,1:,0], Galaxy1_orbit[-1,1:,1], 'o', markersize=1, c='g', label='final position', alpha=0.5, zorder=10)
plt.scatter(Galaxy1_orbit[:,0,0], Galaxy1_orbit[:,0,1], s=10, color='r', label='GC', zorder=10)
plt.xlabel('X [Nbody]')
plt.ylabel('Y [Nbody]')
plt.title('Orbit of Galaxy1')
plt.legend(loc='upper right', fontsize=8)
plt.grid(True, alpha=0.5)
plt.show()

### Stability of the rotation curve and of the surface density

We check that both the rotation curve and the surface density of the galaxy after running one simulation remain stable after a timescale defined as the mean period of the galaxy

In [ ]:
plt.figure(figsize=(5, 5))
n = Galaxy1_Rcyl.shape[0]
snapshots = np.arange(0, n, n//2)
title = ['initial', 'mid', 'final']
for i in range (len(snapshots)):
    plt.scatter(Galaxy1_Rcyl[snapshots[i],:], Galaxy1_Vphi[snapshots[i],:], s=0.5, label=title[i], alpha=0.5)
    
plt.xlabel('$R_{cycl}$')
plt.ylabel('$V_{\phi}$')
plt.title('Tangential velocity Galaxy 1')
plt.legend(loc='lower right')
plt.show()  

In [ ]:
# intial surface density
mid, H, area = surface_density(Galaxy1_Rcyl[0])

# final surface density
mid_end, H_end, area_end = surface_density(Galaxy1_Rcyl[-1])

In [ ]:
# plot the surface density of the initial and final galaxy
plt.figure(figsize=(5,5))
plt.plot(mid, H / area, label='initial')
plt.plot(mid_end, H_end / area_end, label='final')
plt.yscale('log')
plt.xlabel("Rcyl [Nbody]")  
plt.ylabel("Surface Number Density [Nbody]")
plt.title("Surface Density Profile Galaxy 1")
plt.legend()
plt.show()

## Galaxy 2

### Stability of the rotation curve and of the surface density
We do the same check for the other galaxy, so first we run a simulation and then we check the stability of the rotation curve and of the surface density. 

In [ ]:
# compute the orbital period with T = 2*pi*sqrt(a^3/GM)

G = 1.                                                                   # gravitational constant in n-body units
plummer_scale_lenght = 5.                                                # scale lenght of the plummer sphere (softening)

central_mass2 = Galaxy1.mass[0]                                           # central mass of the galaxy
radius = np.sqrt(np.sum((Galaxy2.pos-Galaxy2.pos[0])**2., axis=1))  # radius array

T = 2 * np.pi * np.sqrt(radius**3 / (G * central_mass))        # period of the galaxy using radius array  
T = np.mean(T)                                                           # mean period of the galaxy

print(f'Period of the galaxy (approximately): {T}')

# define the time of simulation and the number of steps

t0 = 0.                         # initial time
tsimulation = 1*T               # time of simulation (one period)
N = int(1e3)                    # number of steps
h = (tsimulation - t0)/N        # time steps

Galaxy2_test = Galaxy2.copy()

Galaxy2_orbit, Galaxy2_stars_vel, Galaxy2_time, Galaxy2_Rcyl, Galaxy2_Vphi = integration_leapfrog(galaxy=Galaxy2_test, h=h, 
                                                                                            tsimulation=tsimulation, t=t0, soft=plummer_scale_lenght, GC=0)

In [ ]:
plt.figure(figsize=(5, 5))
n = Galaxy2_Rcyl.shape[0]
snapshots = np.arange(0, n, n//2)
title = ['initial', 'mid', 'final']
for i in range (len(snapshots)):
    plt.scatter(Galaxy2_Rcyl[snapshots[i],:], Galaxy2_Vphi[snapshots[i],:], s=0.5, label=title[i], alpha=0.5)
    
plt.xlabel('$R_{cycl}$')
plt.ylabel('$V_{\phi}$')
plt.title('Tangential velocity Galaxy 2')
plt.legend(loc='lower right')
plt.show()  

In [ ]:
# intial surface density
mid, H, area = surface_density(Galaxy2_Rcyl[0])

# final surface density
mid_end, H_end, area_end = surface_density(Galaxy2_Rcyl[-1])

In [ ]:
# plot the surface density of the initial and final galaxy
plt.figure(figsize=(5,5))
plt.plot(mid, H / area, label='initial')
plt.plot(mid_end, H_end / area_end, label='final')
plt.yscale('log')
plt.xlabel("Rcyl [Nbody]")  
plt.ylabel("Surface Number Density [Nbody]")
plt.title("Surface Density Profile Galaxy 2")
plt.legend()
plt.show()

----    

## Galaxy encounter

We run a galaxy encounter and then show how the rotation curve and the disc surface profile evolve with time. 
First of all, we make a single class with both galaxies. 

In [ ]:
# Combine Galaxy1 and Galaxy2 into a single Particles class
combined_pos = np.concatenate((Galaxy1.pos, Galaxy2.pos))
combined_vel = np.concatenate((Galaxy1.vel, Galaxy2.vel))
combined_mass = np.concatenate((Galaxy1.mass, Galaxy2.mass))

Combined_Galaxies = Particles(combined_pos, combined_vel, combined_mass)

div = 10000 # first index of the second galaxy

We define another function for the integration `integration_leapfrog_encounter` to differentiate the rescaling for the two galaxies based on the ID of the particles. 

In [ ]:
def integration_leapfrog_encounter(galaxy: Particles, h: float, tsimulation: float, t: float, soft: float):
    N = int((tsimulation - t) / h)  # number of time steps
    num_particles = len(galaxy.mass)  # number of particles in the galaxy
    path = np.empty((N, num_particles, 3))  # array to store the position of the galaxy
    velocity = np.empty((N, num_particles, 3))  # array to store the velocity of the galaxy
    time = np.empty(N)
    timestep = np.empty(N)
    
    R_cyl, _, V_phi = rotation_curve_rescaled(galaxy, GC=np.where(galaxy.ID < (len(galaxy.ID)//2), 0, len(galaxy.ID)//2))
    R_cyl = [R_cyl]
    V_phi = [V_phi]

    i = 0
    while t < tsimulation:
        updated_galaxy, _,  updated_acc, _, _ = integrator_leapfrog(particles=galaxy, tstep=h, acceleration_estimator=acceleration_pyfalcon, softening=soft)
        path[i] = updated_galaxy.pos
        velocity[i] = updated_galaxy.vel
            
        R, z, V = rotation_curve_rescaled(updated_galaxy, GC=np.where(updated_galaxy.ID < (len(updated_galaxy.ID)//2), 0, len(updated_galaxy.ID)//2))

        timestep[i] = h
        t += h
        time[i] = t

        R_cyl.append(R)
        V_phi.append(V) 

        i += 1

    R_cyl = np.array(R_cyl)
    V_phi = np.array(V_phi)

    return path, velocity, time, R_cyl, V_phi

Then we run the simulation for the encounter using $T=210$ as time of the simulation.

In [ ]:
# just keep the same tsimulation and N as before
G = 1.                                                                   # gravitational constant in n-body units
T_sim = 210
plummer_scale_lenght = 5.                                                # scale lenght of the plummer sphere (softening)
t0 = 0.
tsimulation = T_sim
N = int(1e3)
h = (tsimulation - t0)/N

Combined_Galaxies_test = Combined_Galaxies.copy()

Combined_Galaxies_orbit, Combined_Galaxies_stars_vel, Combined_Galaxies_time, Combined_Galaxies_Rcyl, Combined_Galaxies_Vphi = integration_leapfrog_encounter(galaxy=Combined_Galaxies_test, h=h, 
                                                                                                                                                                            tsimulation=tsimulation, t=t0, soft=plummer_scale_lenght)

### Plot of the whole evolution
We make a plot showing the whole evolution of the galaxies, namely the trajectories of the trackers and of the galactic centers. We display even the initial and final positions of the galaxies. 

In [ ]:
plt.figure(figsize=(8,8))

# stars trajectory
plt.scatter(Combined_Galaxies_orbit[:,1:div,0], Combined_Galaxies_orbit[:,1:div,1], s=0.005, color='lightblue',  alpha=0.05)
plt.scatter(Combined_Galaxies_orbit[:,(div+1):,0],  Combined_Galaxies_orbit[:,(div+1):,1],  s=0.005, color='lightgreen', alpha=0.05)

# initial and final position of galaxy 1
plt.plot(Combined_Galaxies_orbit[0,1:div,0],  Combined_Galaxies_orbit[0,1:div,1],  'o', markersize=2, c='cornflowerblue', label='initial position of Gal1', alpha=0.5)
plt.plot(Combined_Galaxies_orbit[-1,1:div,0], Combined_Galaxies_orbit[-1,1:div,1], '*', markersize=3, c='midnightblue', label='final position of Gal1',   alpha=0.5)

# initial and final position of galaxy 2
plt.plot(Combined_Galaxies_orbit[0,(div+1):,0],  Combined_Galaxies_orbit[0,(div+1):,1],  'o', markersize=2, c='limegreen', label='initial position of Gal2', alpha=0.5)
plt.plot(Combined_Galaxies_orbit[-1,(div+1):,0], Combined_Galaxies_orbit[-1,(div+1):,1], '*', markersize=3, c='darkgreen', label='final position of Gal2',   alpha=0.5)

# initial and final position of GC1
plt.plot(Combined_Galaxies_orbit[0,0,0],  Combined_Galaxies_orbit[0,0,1],  'o', markersize=4, color='r', label='GC 1', zorder=10)
plt.plot(Combined_Galaxies_orbit[-1,0,0], Combined_Galaxies_orbit[-1,0,1], 'o', markersize=4, color='r',               zorder=10)

# initial and final position of GC2
plt.plot(Combined_Galaxies_orbit[0,div,0],  Combined_Galaxies_orbit[0,div,1],  'o', markersize=4, color='orange', label='GC 2', zorder=10)
plt.plot(Combined_Galaxies_orbit[-1,div,0], Combined_Galaxies_orbit[-1,div,1], 'o', markersize=4, color='orange',               zorder=10)

# trajectory of GC1
plt.scatter(Combined_Galaxies_orbit[:,0,0],    Combined_Galaxies_orbit[:,0,1],    s=0.05, color='r',      zorder=10, alpha=0.5)
# trajectory of GC2
plt.scatter(Combined_Galaxies_orbit[:,div,0], Combined_Galaxies_orbit[:,div,1], s=0.05, color='orange', zorder=10, alpha=0.5)

plt.xlabel('X [Nbody]')
plt.ylabel('Y [Nbody]')
plt.title('Galaxies Encounter')
plt.legend(loc='lower right', fontsize=8)
plt.grid(True, alpha=0.5)
plt.show()

### Final situation
We make a plot showing the final configuration of the galaxies after the encounter. 

In [ ]:
plt.subplots(1,3,figsize=(12,4), sharey=True)

plt.subplot(1, 3, 1)
# final position of galaxy 1
plt.plot(Combined_Galaxies_orbit[-1,1:div,0], Combined_Galaxies_orbit[-1,1:div,1], 'o', markersize=1, c='midnightblue', label='final position of Gal1',   alpha=0.5)
# position of GC1
plt.plot(Combined_Galaxies_orbit[-1,0,0], Combined_Galaxies_orbit[-1,0,1], 'o', markersize=4, color='r', label='GC 1', zorder=10)
# final position of galaxy 2
plt.plot(Combined_Galaxies_orbit[-1,(div+1):,0], Combined_Galaxies_orbit[-1,(div+1):,1], 'o', markersize=1, c='g', label='final position of Gal2',   alpha=0.5)
# position of GC2
plt.plot(Combined_Galaxies_orbit[-1,div,0], Combined_Galaxies_orbit[-1,div,1], 'o', markersize=4, color='orange', label='GC 2', zorder=10)

plt.xlabel('X [Nbody]')
plt.ylabel('Y [Nbody]')
plt.title('Final position of the Galaxies')
plt.legend(loc='lower right', fontsize=8)
plt.grid(True, alpha=0.5)

plt.subplot(1, 3, 2)
# final position of galaxy 1
plt.plot(Combined_Galaxies_orbit[-1,1:div,0], Combined_Galaxies_orbit[-1,1:div,1], 'o', markersize=1, c='midnightblue', label='final position of Gal1',   alpha=0.5)
# position of GC1
plt.plot(Combined_Galaxies_orbit[-1,0,0], Combined_Galaxies_orbit[-1,0,1], 'o', markersize=4, color='r', label='GC 1', zorder=10)

plt.xlabel('X [Nbody]')
plt.title('Galaxy 1')
plt.legend(loc='lower right', fontsize=8)
plt.grid(True, alpha=0.5)

plt.subplot(1, 3, 3)
# final position of galaxy 2   
plt.plot(Combined_Galaxies_orbit[-1,(div+1):,0], Combined_Galaxies_orbit[-1,(div+1):,1], 'o', markersize=1, c='g', label='final position of Gal2',   alpha=0.5)
# position of GC2
plt.plot(Combined_Galaxies_orbit[-1,div,0], Combined_Galaxies_orbit[-1,div,1], 'o', markersize=4, color='orange', label='GC 2', zorder=10)

plt.xlabel('X [Nbody]')
plt.title('Galaxy 2')
plt.legend(loc='lower right', fontsize=8)
plt.grid(True, alpha=0.5)

plt.tight_layout()
plt.show()

### Evolution of the rotation curve
Check how the rotation curves of the galaxies evolve after the encounter.  

In [ ]:
plt.subplots(1,3,figsize=(12,4), sharey=True)

plt.subplot(1, 3, 1)
plt.scatter(Combined_Galaxies_Rcyl[-1,1:div], Combined_Galaxies_Vphi[-1,1:div], s=0.5, color='r', label='final', alpha=0.5)
plt.scatter(Combined_Galaxies_Rcyl[0,1:div], Combined_Galaxies_Vphi[0,1:div], s=0.5, color='b', label='initial', alpha=0.5)
plt.legend(loc='best')
plt.xlabel('$R_{cycl}$')
plt.ylabel('$V_{\phi}$')
plt.title('Rotation curve Galaxy 1')

plt.subplot(1, 3, 2)
plt.scatter(Combined_Galaxies_Rcyl[-1,(div+1):], Combined_Galaxies_Vphi[-1,(div+1):], s=0.5, color='g', label='final', alpha=0.5)
plt.scatter(Combined_Galaxies_Rcyl[0,(div+1):], Combined_Galaxies_Vphi[0,(div+1):], s=0.5, color='b', label='initial', alpha=0.5)
plt.legend(loc='best')
plt.xlabel('$R_{cycl}$')
plt.title('Rotation curve Galaxy 2')

plt.subplot(1, 3, 3)
plt.scatter(Combined_Galaxies_Rcyl[-1,1:div], Combined_Galaxies_Vphi[-1,1:div], s=0.5, color='r', label='final G1', alpha=0.5)
plt.scatter(Combined_Galaxies_Rcyl[-1,(div+1):], Combined_Galaxies_Vphi[-1,(div+1):], s=0.5, color='g', label='final G2', alpha=0.5)
plt.scatter(Combined_Galaxies_Rcyl[0,1:div], Combined_Galaxies_Vphi[0,1:div], s=0.5, color='b', label='initial G1 and G2', alpha=0.5)
plt.legend(loc='best')
plt.xlabel('$R_{cycl}$')
plt.title('Rotation curve of both Galaxies')

plt.tight_layout()
plt.show()

### Evolution of the surface density
Check the evolution of the surface density of both galaxies after the encounter.

In [ ]:
# intial surface density G1
mid_in_1, H_in_1, area_in_1 = surface_density(Combined_Galaxies_Rcyl[0,:div])

# final surface density G1
mid_end_1, H_end_1, area_end_1 = surface_density(Combined_Galaxies_Rcyl[-1,:div])

# intial surface density G2 
mid_in_2, H_in_2, area_in_2 = surface_density(Combined_Galaxies_Rcyl[0,div:])

# final surface density G2
mid_end_2, H_end_2, area_end_2 = surface_density(Combined_Galaxies_Rcyl[-1,div:])

## Final plots after the encounter
Let's make a single plot containing the final situation of the galaxies, the evolution of their rotation curve and the one of their surface density. 

In [ ]:
plt.subplots(3,3,figsize=(12,12))

plt.subplot(3, 3, 1)
# final position of galaxy 1
plt.plot(Combined_Galaxies_orbit[-1,1:div,0], Combined_Galaxies_orbit[-1,1:div,1], 'o', markersize=1, c='midnightblue', label='final position of Gal1',   alpha=0.5)
# position of GC1
plt.plot(Combined_Galaxies_orbit[-1,0,0], Combined_Galaxies_orbit[-1,0,1], 'o', markersize=4, color='r', label='GC 1', zorder=10)
# final position of galaxy 2
plt.plot(Combined_Galaxies_orbit[-1,(div+1):,0], Combined_Galaxies_orbit[-1,(div+1):,1], 'o', markersize=1, c='g', label='final position of Gal2',   alpha=0.5)
# position of GC2
plt.plot(Combined_Galaxies_orbit[-1,div,0], Combined_Galaxies_orbit[-1,div,1], 'o', markersize=4, color='orange', label='GC 2', zorder=10)

plt.xlabel('X [Nbody]')
plt.ylabel('Y [Nbody]')
plt.title('Final position of the Galaxies')
plt.legend(loc='lower right', fontsize=8)

plt.subplot(3, 3, 2)
# final position of galaxy 1
plt.plot(Combined_Galaxies_orbit[-1,1:div,0], Combined_Galaxies_orbit[-1,1:div,1], 'o', markersize=1, c='midnightblue', label='final position of Gal1',   alpha=0.5)
# position of GC1
plt.plot(Combined_Galaxies_orbit[-1,0,0], Combined_Galaxies_orbit[-1,0,1], 'o', markersize=4, color='r', label='GC 1', zorder=10)

plt.xlabel('X [Nbody]')
plt.ylabel('Y [Nbody]')
plt.title('Galaxy 1')
plt.legend(loc='lower right', fontsize=8)

plt.subplot(3, 3, 3)
# final position of galaxy 2   
plt.plot(Combined_Galaxies_orbit[-1,(div+1):,0], Combined_Galaxies_orbit[-1,(div+1):,1], 'o', markersize=1, c='g', label='final position of Gal2',   alpha=0.5)
# position of GC2
plt.plot(Combined_Galaxies_orbit[-1,div,0], Combined_Galaxies_orbit[-1,div,1], 'o', markersize=4, color='orange', label='GC 2', zorder=10)

plt.xlabel('X [Nbody]')
plt.ylabel('Y [Nbody]')
plt.title('Galaxy 2')
plt.legend(loc='lower right', fontsize=8)

plt.subplot(3, 3, 5)
plt.scatter(Combined_Galaxies_Rcyl[-1,1:div], Combined_Galaxies_Vphi[-1,1:div], s=0.5, color='r', label='final', alpha=0.5)
plt.scatter(Combined_Galaxies_Rcyl[0,1:div], Combined_Galaxies_Vphi[0,1:div], s=0.5, color='b', label='initial', alpha=0.5)
plt.legend(loc='best', fontsize=8)
plt.xlabel('$R_{cycl}$')
plt.ylabel('$V_{\phi}$')
plt.title('Rotation curve Galaxy 1')

plt.subplot(3, 3, 6)
plt.scatter(Combined_Galaxies_Rcyl[-1,(div+1):], Combined_Galaxies_Vphi[-1,(div+1):], s=0.5, color='g', label='final', alpha=0.5)
plt.scatter(Combined_Galaxies_Rcyl[0,(div+1):], Combined_Galaxies_Vphi[0,(div+1):], s=0.5, color='b', label='initial', alpha=0.5)
plt.legend(loc='best', fontsize=8)
plt.xlabel('$R_{cycl}$')
plt.ylabel('$V_{\phi}$')
plt.title('Rotation curve Galaxy 2')

plt.subplot(3, 3, 4)
plt.scatter(Combined_Galaxies_Rcyl[-1,1:div], Combined_Galaxies_Vphi[-1,1:div], s=0.5, color='r', label='final G1', alpha=0.5)
plt.scatter(Combined_Galaxies_Rcyl[-1,(div+1):], Combined_Galaxies_Vphi[-1,(div+1):], s=0.5, color='g', label='final G2', alpha=0.5)
plt.scatter(Combined_Galaxies_Rcyl[0,1:div], Combined_Galaxies_Vphi[0,1:div], s=0.5, color='b', label='initial G1 and G2', alpha=0.5)
plt.legend(loc='best', fontsize=8)
plt.xlabel('$R_{cycl}$')
plt.ylabel('$V_{\phi}$')
plt.title('Rotation curve (G1 and G2)')

plt.subplot(3, 3, 7)
plt.plot(mid_in_1, H_in_1 / area_in_1, label='initial G1 and G2', c='b')
plt.plot(mid_end_1, H_end_1 / area_end_1, label='final G1', c='r')
plt.plot(mid_end_2, H_end_2 / area_end_2, label='final G2', c='g')
plt.yscale('log')
plt.xlabel("Rcyl [Nbody]")  
plt.ylabel("Surface Number Density [Nbody]")
plt.legend(loc='best', fontsize=8)
plt.title("Surface Density Profile (G1 and G2)")

plt.subplot(3, 3, 8)
plt.plot(mid_in_1, H_in_1 / area_in_1, label='initial', c='b')
plt.plot(mid_end_1, H_end_1 / area_end_1, label='final', c='r')
plt.yscale('log')
plt.xlabel("Rcyl [Nbody]")  
plt.ylabel("Surface Number Density [Nbody]")
plt.legend(loc='best', fontsize=8)
plt.title("Surface Density Profile Galaxy 1")

plt.subplot(3, 3, 9)
plt.plot(mid_in_2, H_in_2 / area_in_2, label='initial', c='b')
plt.plot(mid_end_2, H_end_2 / area_end_2, label='final', c='g')
plt.yscale('log')
plt.xlabel("Rcyl [Nbody]")  
plt.ylabel("Surface Number Density [Nbody]")
plt.legend(loc='best', fontsize=8)
plt.title("Surface Density Profile Galaxy 2")


plt.tight_layout()
plt.show()